In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Dropout,Conv1D,MaxPooling1D,initializers,Input
from keras.models import Model
from keras import optimizers,regularizers
from keras.callbacks import History,EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras import regularizers
import nltk
from nltk import pos_tag
from collections import namedtuple
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from gensim.models import doc2vec
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score,f1_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from keras import backend as K
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.utils import shuffle, class_weight
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
with open('task2data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
print(list(data.columns.values))

['sentences', 'article', 'N_sentence', 'is_propaganda', 'sadness', 'joy', 'fear', 'disgust', 'anger']


In [4]:
#data = shuffle(data)

In [4]:
def tokenise(x):
 x=x.lower()
 tokenizer = RegexpTokenizer(r'\w+')
 x=(tokenizer.tokenize(x))
 filtered_words =([word for word in x if word not in stopwords.words('english')])
 filtered_words=" ".join(filtered_words)
 st = PorterStemmer()
 stemmed_words = ''.join(([st.stem(word) for word in filtered_words]))
 return stemmed_words

In [5]:
data["new_sentences"] = data["sentences"].apply(lambda x: tokenise(x))

In [6]:
tag_values=[]
docs=[]
i=0
y= np.array([])
for index, row in data.iterrows():
 w=row["new_sentences"].split()
 word_value=row["new_sentences"].split(" ")
 t1=[i]
 all_tags=nltk.pos_tag(row["sentences"].split())
 tag_values.append(all_tags)
 docs.append(TaggedDocument(words=w, tags=t1))
 if(row["is_propaganda"]=="propaganda"):
        y=np.append(y, 0)
 else:
    y=np.append(y, 1)
 i+=1

In [7]:
#including grammatical features nouns, pronouns, etc
pos_tags=['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
tagm = [[sum(sen,()).count(tag) for tag in pos_tags] for sen in tag_values]
tagm=np.array(tagm)
print(tagm.shape)

(14263, 35)


In [8]:
size=10
model = models.Doc2Vec(vector_size=size, window=3, min_count=2, workers=4, epochs=20)
model.build_vocab(docs)

In [9]:
model.train(docs, total_examples=model.corpus_count, epochs=model.iter)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [10]:
y=y.reshape(-1, 1)
x=np.zeros((y.shape[0], size))
for i in range(y.shape[0]):
    x[i]=model[i]

In [11]:
count_0=np.sum(y==0)
count_1=np.sum(y==1)

print(count_0/y.shape[0])
print(count_1/y.shape[0])

0.27609899740587535
0.7239010025941247


In [12]:
values=np.array(data.iloc[:,4:9])
values2=np.array(data.iloc[:,2:3])
x=np.append(x, values, axis=1)
x=np.append(x, values2,axis=1)
x=np.append(x, tagm,axis=1)
print(x.shape)

(14263, 51)


In [13]:
split_index=int(y.shape[0]*0.8)
y_train=y[0:split_index, :]
x_train=x[0:split_index,:]
y_val=y[split_index:,:]
x_val=x[split_index:,:]

print(x_train.shape)
print(x_val.shape)

data_train= data.iloc[0:split_index,0:1]
data_val=data.iloc[split_index:,0:1]

(11410, 51)
(2853, 51)


In [16]:
count_0_train=np.sum(y_train==0)
count_1_train=np.sum(y_train==1)

print(count_0_train/y_train.shape[0])
print(count_1_train/y_train.shape[0])

0.28089395267309375
0.7191060473269062


In [17]:
count_0=np.sum(y_val==0)
count_1=np.sum(y_val==1)

print(count_0/y_val.shape[0])
print(count_1/y_val.shape[0])

0.25692253767963547
0.7430774623203645


In [37]:
#Using Logistic Regression as baseline
logReg = LogisticRegression()
logReg.fit(x_train, y_train)


y_pred_train=logReg.predict(x_train).reshape(-1,1)
y_pred_val=logReg.predict(x_val).reshape(-1,1)

train_recall=recall_score(y_train, y_pred_train, average="macro")
train_precision=precision_score(y_train,y_pred_train,average="macro")
train_f1=f1_score(y_train,y_pred_train,average="macro")

val_recall=recall_score(y_val, y_pred_val, average="macro")
val_precision=precision_score(y_val,y_pred_val,average="macro")
val_f1=f1_score(y_val,y_pred_val,average="macro")

print("For training \n")
print("Precision: "+str(train_precision))
print("Recall: "+str(train_recall))
print("F1: "+str(train_f1))

print("For validation \n")
print("Precision: "+str(val_precision))
print("Recall: "+str(val_recall))
print("F1: "+str(val_f1))


For training 

Precision: 0.680821581280176
Recall: 0.5717098226890684
F1: 0.5653396669692063
For validation 

Precision: 0.6653280330537124
Recall: 0.5679798707817447
F1: 0.5656317149886332


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [54]:
#Using K-Nearest Neighbours
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train, y_train)
y_pred_train=knn.predict(x_train).reshape(-1,1)
y_pred_val=knn.predict(x_val).reshape(-1,1)

train_recall=recall_score(y_train, y_pred_train, average="macro")
train_precision=precision_score(y_train,y_pred_train,average="macro")
train_f1=f1_score(y_train,y_pred_train,average="macro")

val_recall=recall_score(y_val, y_pred_val, average="macro")
val_precision=precision_score(y_val,y_pred_val,average="macro")
val_f1=f1_score(y_val,y_pred_val,average="macro")

print("For training \n")
print("Precision: "+str(train_precision))
print("Recall: "+str(train_recall))
print("F1: "+str(train_f1))

print("For validation \n")
print("Precision: "+str(val_precision))
print("Recall: "+str(val_recall))
print("F1: "+str(val_f1))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


For training 

Precision: 0.8150979276287147
Recall: 0.8853065128089584
F1: 0.8219652103978099
For validation 

Precision: 0.5541095683137556
Recall: 0.5690381348297253
F1: 0.5440381917889026


In [57]:
#Using Random Forest Classifier
rfc=RandomForestClassifier(n_estimators=3, bootstrap = True, max_features = 'sqrt')
rfc.fit(x_train, y_train)
y_pred_train=rfc.predict(x_train).reshape(-1,1)
y_pred_val=rfc.predict(x_val).reshape(-1,1)

train_recall=recall_score(y_train, y_pred_train, average="macro")
train_precision=precision_score(y_train,y_pred_train,average="macro")
train_f1=f1_score(y_train,y_pred_train,average="macro")

val_recall=recall_score(y_val, y_pred_val, average="macro")
val_precision=precision_score(y_val,y_pred_val,average="macro")
val_f1=f1_score(y_val,y_pred_val,average="macro")

print("For training \n")
print("Precision: "+str(train_precision))
print("Recall: "+str(train_recall))
print("F1: "+str(train_f1))

print("For validation \n")
print("Precision: "+str(val_precision))
print("Recall: "+str(val_recall))
print("F1: "+str(val_f1))

For training 

Precision: 0.9335147655462391
Recall: 0.9190258213619221
F1: 0.9258884661779716
For validation 

Precision: 0.5775846133343651
Recall: 0.5696832608303946
F1: 0.572331894101405


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [42]:

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true,y_pred):
   precisions = precision(y_true, y_pred)
   recalls = recall(y_true, y_pred)
   return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [43]:
y_train=y_train.reshape((y_train.shape[0],))
print(y_train.shape)
class_weights_values= class_weight.compute_class_weight('balanced',np.unique(y_train), y_train)
y_train=y_train.reshape(-1, 1)

(11410,)


In [44]:
model = Sequential()
model.add(Dense(175, input_dim=x_train.shape[1], activation='relu',activity_regularizer=regularizers.l2(0.01)))
model.add(Dense(175, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
sgd=optimizers.SGD(lr=0.080, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['mse','acc',f1,precision,recall])
train=model.fit(x_train, y_train,epochs=10, verbose=1,class_weight=class_weights_values,batch_size=x_train.shape[0])
train_model=model.evaluate(x_train,y_train,verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
11410/11410 [==============================] - 0s 30us/step - loss: 482063.6875 - mean_squared_error: 0.5810 - acc: 0.2931 - f1: 0.0641 - precision: 0.6683 - recall: 0.0336
Epoch 2/10
11410/11410 [==============================] - 0s 4us/step - loss: 0.6835 - mean_squared_error: 0.2418 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 - recall: 1.0000
Epoch 3/10
11410/11410 [==============================] - 0s 4us/step - loss: 0.6721 - mean_squared_error: 0.2369 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 - recall: 1.0000
Epoch 4/10
11410/11410 [==============================] - 0s 4us/step - loss: 0.6626 - mean_squared_error: 0.2327 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 - recall: 1.0000
Epoch 5/10
11410/11410 [==============================] - 0s 4us/step - loss: 0.6544 - mean_squared_error: 0.2289 - acc: 0.7191 - f1: 0.8366 - precision: 0.7191 -

In [45]:
#For training
print("For training \n")
print("Precision: "+str(train_model[4]))
print("Recall: "+str(train_model[5]))
print("F1: "+str(train_model[3]))
print("Accuracy: "+str(train_model[2]))

For training 

Precision: 0.7191060473373541
Recall: 1.0
F1: 0.82643346677842
Accuracy: 0.7191060473373541


In [46]:
val_model=model.evaluate(x_val,y_val,verbose=2)

In [47]:
print("For validation \n")
print("Precision: "+str(val_model[4]))
print("Recall: "+str(val_model[5]))
print("F1: "+str(val_model[3]))
print("Accuracy: "+str(val_model[2]))

model1_prec=val_model[4]
model2_recall=val_model[5]

For validation 

Precision: 0.7430774623203645
Recall: 1.0
F1: 0.8451232763077141
Accuracy: 0.7430774623203645


In [48]:
#using LSTM
new_x_train=x_train
new_x_train= new_x_train.reshape((new_x_train.shape[0],1,new_x_train.shape[1]))
print(new_x_train.shape)

(11410, 1, 51)


In [49]:
new_x_val=x_val
new_x_val= new_x_val.reshape((new_x_val.shape[0],1,new_x_val.shape[1]))
print(new_x_val.shape)

(2853, 1, 51)


In [123]:
model = Sequential()
model.add(LSTM(175,input_shape=(1,x.shape[1]),return_sequences=True,activation='relu'))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(175, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(1, activation='sigmoid'))
#es=EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=2, mode='auto')
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['mse','acc',f1,precision,recall])
train=model.fit(new_x_train, y_train,epochs=60, verbose=2,class_weight=class_weights_values,batch_size=x_train.shape[0],validation_data=(new_x_val, y_val))
train_model=model.evaluate(new_x_train,y_train,verbose=2)

Train on 11410 samples, validate on 2853 samples
Epoch 1/60
 - 2s - loss: 1.3939 - mean_squared_error: 0.3663 - acc: 0.3975 - f1: 0.3958 - precision: 0.7095 - recall: 0.2745 - val_loss: 0.9217 - val_mean_squared_error: 0.3237 - val_acc: 0.4791 - val_f1: 0.5900 - val_precision: 0.7108 - val_recall: 0.5042
Epoch 2/60
 - 0s - loss: 1.1471 - mean_squared_error: 0.3231 - acc: 0.4938 - f1: 0.5842 - precision: 0.7136 - recall: 0.4945 - val_loss: 0.7095 - val_mean_squared_error: 0.2549 - val_acc: 0.6200 - val_f1: 0.7517 - val_precision: 0.7306 - val_recall: 0.7741
Epoch 3/60
 - 0s - loss: 0.9488 - mean_squared_error: 0.2847 - acc: 0.5767 - f1: 0.6940 - precision: 0.7226 - recall: 0.6676 - val_loss: 0.6105 - val_mean_squared_error: 0.2093 - val_acc: 0.7392 - val_f1: 0.8499 - val_precision: 0.7424 - val_recall: 0.9939
Epoch 4/60
 - 0s - loss: 0.8846 - mean_squared_error: 0.2679 - acc: 0.6215 - f1: 0.7464 - precision: 0.7202 - recall: 0.7745 - val_loss: 0.5853 - val_mean_squared_error: 0.1975 - v

Epoch 33/60
 - 0s - loss: 0.6094 - mean_squared_error: 0.2057 - acc: 0.7055 - f1: 0.8235 - precision: 0.7236 - recall: 0.9555 - val_loss: 0.5545 - val_mean_squared_error: 0.1847 - val_acc: 0.7434 - val_f1: 0.8527 - val_precision: 0.7437 - val_recall: 0.9991
Epoch 34/60
 - 0s - loss: 0.6072 - mean_squared_error: 0.2051 - acc: 0.7089 - f1: 0.8255 - precision: 0.7254 - recall: 0.9576 - val_loss: 0.5539 - val_mean_squared_error: 0.1845 - val_acc: 0.7424 - val_f1: 0.8520 - val_precision: 0.7434 - val_recall: 0.9976
Epoch 35/60
 - 0s - loss: 0.6100 - mean_squared_error: 0.2051 - acc: 0.7080 - f1: 0.8251 - precision: 0.7247 - recall: 0.9578 - val_loss: 0.5525 - val_mean_squared_error: 0.1840 - val_acc: 0.7427 - val_f1: 0.8521 - val_precision: 0.7437 - val_recall: 0.9976
Epoch 36/60
 - 0s - loss: 0.6049 - mean_squared_error: 0.2039 - acc: 0.7093 - f1: 0.8266 - precision: 0.7238 - recall: 0.9634 - val_loss: 0.5533 - val_mean_squared_error: 0.1843 - val_acc: 0.7441 - val_f1: 0.8528 - val_precisi

In [124]:
print("For training \n")
print("Precision: "+str(train_model[4]))
print("Recall: "+str(train_model[5]))
print("F1: "+str(train_model[3]))
print("Accuracy: "+str(train_model[2]))

For training 

Precision: 0.7242916825254375
Recall: 0.9907047804983743
F1: 0.8270446840866317
Accuracy: 0.7234005258649614


In [125]:
val_model=model.evaluate(new_x_val,y_val,verbose=2)

In [126]:
print("For validation \n")
print("Precision: "+str(val_model[4]))
print("Recall: "+str(val_model[5]))
print("F1: "+str(val_model[3]))
print("Accuracy: "+str(val_model[2]))

model2_prec=val_model[4]
model2_recall=val_model[5]

For validation 

Precision: 0.7481575139229899
Recall: 0.989601422726711
F1: 0.8444051558704824
Accuracy: 0.7441289870311952


In [133]:
vocabulary_size = 1200
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data_train['sentences'])
sequences = tokenizer.texts_to_sequences(data_train['sentences'])
data_words_train = pad_sequences(sequences, maxlen=50)
#data_words_train=np.append(data_words_train, tagm,axis=1)

In [135]:
#using LSTM for word embeddings
model = Sequential()
model.add(Embedding(1200, 100, input_length=50))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(175, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
#es=EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=2, mode='auto')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['mse','acc',f1,precision,recall])
train=model.fit(data_words_train, y_train,epochs=30, verbose=2,class_weight=class_weights_values,batch_size=x_train.shape[0])
train_model=model.evaluate(data_words_train,y_train,verbose=2)

Epoch 1/30
 - 3s - loss: 0.6863 - mean_squared_error: 0.2466 - acc: 0.5735 - f1: 0.6872 - precision: 0.7271 - recall: 0.6514
Epoch 2/30
 - 1s - loss: 0.7852 - mean_squared_error: 0.2345 - acc: 0.7200 - f1: 0.8369 - precision: 0.7200 - recall: 0.9990
Epoch 3/30
 - 1s - loss: 0.6080 - mean_squared_error: 0.2084 - acc: 0.7243 - f1: 0.8380 - precision: 0.7255 - recall: 0.9918
Epoch 4/30
 - 1s - loss: 0.5732 - mean_squared_error: 0.1933 - acc: 0.7232 - f1: 0.8376 - precision: 0.7244 - recall: 0.9928
Epoch 5/30
 - 2s - loss: 0.5705 - mean_squared_error: 0.1922 - acc: 0.7247 - f1: 0.8382 - precision: 0.7259 - recall: 0.9917
Epoch 6/30
 - 1s - loss: 0.5664 - mean_squared_error: 0.1907 - acc: 0.7258 - f1: 0.8387 - precision: 0.7267 - recall: 0.9916
Epoch 7/30
 - 1s - loss: 0.5647 - mean_squared_error: 0.1899 - acc: 0.7266 - f1: 0.8390 - precision: 0.7276 - recall: 0.9907
Epoch 8/30
 - 1s - loss: 0.5610 - mean_squared_error: 0.1886 - acc: 0.7272 - f1: 0.8393 - precision: 0.7279 - recall: 0.9911


In [137]:
print("For training \n")
print("Precision: "+str(train_model[4]))
print("Recall: "+str(train_model[5]))
print("F1: "+str(train_model[3]))
print("Accuracy: "+str(train_model[2]))

For training 

Precision: 0.7594832291632342
Recall: 0.9853250294584229
F1: 0.8501454054734667
Accuracy: 0.7707274320353341


In [138]:
vocabulary_size = 1200
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data_val['sentences'])
sequences = tokenizer.texts_to_sequences(data_val['sentences'])
data_words_val = pad_sequences(sequences, maxlen=50)


In [139]:
val_model=model.evaluate(data_words_val,y_val,verbose=2)

In [140]:
print("For validation \n")
print("Precision: "+str(val_model[4]))
print("Recall: "+str(val_model[5]))
print("F1: "+str(val_model[3]))
print("Accuracy: "+str(val_model[2]))
model1_prec=val_model[4]
model2_recall=val_model[5]

For validation 

Precision: 0.7501297164205664
Recall: 0.9583619482426153
F1: 0.8333367870182394
Accuracy: 0.7308096740273397


In [143]:
#Using CNN for word embeddings
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
model_conv.add(Dropout(0.5))
model_conv.add(Conv1D(175, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(175,activation='relu'))
model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy',f1,precision,recall])
train=model_conv.fit(data_words_train, y_train, epochs=3,class_weight=class_weights_values)
train_model=model_conv.evaluate(data_words_train,y_train,verbose=2)

Epoch 1/3
11410/11410 [==============================] - 12s 1ms/step - loss: 0.5684 - acc: 0.7223 - f1: 0.8325 - precision: 0.7298 - recall: 0.9769
Epoch 2/3
11410/11410 [==============================] - 11s 944us/step - loss: 0.5296 - acc: 0.7490 - f1: 0.8408 - precision: 0.7645 - recall: 0.9426
Epoch 3/3
11410/11410 [==============================] - 11s 931us/step - loss: 0.5055 - acc: 0.7656 - f1: 0.8500 - precision: 0.7788 - recall: 0.9435


In [144]:
print("For training \n")
print("Precision: "+str(train_model[3]))
print("Recall: "+str(train_model[4]))
print("F1: "+str(train_model[2]))
print("Accuracy: "+str(train_model[1]))

For training 

Precision: 0.7596727901127754
Recall: 0.9714273364161525
F1: 0.8459418138634417
Accuracy: 0.7670464504402421


In [145]:
val_model=model_conv.evaluate(data_words_val,y_val,verbose=2)

In [146]:
print("For validation \n")
print("Precision: "+str(val_model[3]))
print("Recall: "+str(val_model[4]))
print("F1: "+str(val_model[2]))
print("Accuracy: "+str(val_model[1]))
model1_prec=val_model[3]
model2_recall=val_model[4]

For validation 

Precision: 0.7497233782907641
Recall: 0.9513095911870368
F1: 0.830600403586146
Accuracy: 0.727304591657904
